In [1]:
import requests
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
load_dotenv()

gameName = "BakutoPhinn"
tagLine = "NA1"
api_key = os.environ.get("riot_api")

In [2]:
api_key

'RGAPI-03dfa5b4-5e54-4cf6-b7e8-1887fa58b79a'

In [4]:
def get_puuid_ign(gameName, tagLine, api_key):
    gameNamev2 = ""
    for i in gameName:
        if (i==' '):
            gameNamev2 += "%20"
        else:
            gameNamev2 += i
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameNamev2}/{tagLine}?api_key={api_key}"
    print(link)
    response = requests.get(link)
    return response.json()

def get_puuid_sID(sID, api_key):
    link = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{sID}?api_key={api_key}"
    response = requests.get(link)
    return response.json()['puuid']

def get_name_and_tag(puuid, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
    response = requests.get(link)
    return [response.json()['gameName'], response.json()['tagLine']]
# must use strings challengerleagues, grandmasterleagues, or masterleagues for league input
def get_league(league, api_key):
    link = f"https://na1.api.riotgames.com/lol/league/v4/{league}/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(link).json()

In [5]:
get_puuid_ign("BakutoPhinn", "NA1", api_key)

https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/BakutoPhinn/NA1?api_key=RGAPI-03dfa5b4-5e54-4cf6-b7e8-1887fa58b79a


{'puuid': 'HkKgb4Qh01HHrJaU-wW5OCD2sp6tnDH4SuB75lT4vmwHZnWXjpkEnw3VIQpPMWXOpRG64EzQLmEixA',
 'gameName': 'BakutoPhinn',
 'tagLine': 'NA1'}

In [42]:
def get_ladder():
    chall_response = get_league("challengerleagues", api_key)
    chall_df = pd.DataFrame(chall_response['entries'])
    chall_df = chall_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    gm_response = get_league("grandmasterleagues", api_key)
    gm_df = pd.DataFrame(gm_response['entries'])
    gm_df = gm_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    masters_response = get_league("masterleagues", api_key)
    masters_df = pd.DataFrame(masters_response['entries'])
    masters_df = masters_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    ladder = pd.concat([chall_df, gm_df, masters_df])
    ladder = ladder.drop(columns='rank')
    ladder = ladder.reset_index()
    ladder = ladder.rename(columns={"index":"Rank"})
    ladder["Rank"] = ladder["Rank"]+1
    return ladder

In [43]:
def get_match_history(puuid, api_key, start=0, count=20, type=""):
    if type != "":
        link = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={count}&api_key={api_key}"
    else:
        link = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={api_key}"
    return requests.get(link).json()

def get_match_data(matchID, api_key):
    link = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}?api_key={api_key}"
    return requests.get(link).json()

def get_usernames(matchID, api_key):
    data = get_match_data(matchID, api_key)
    usernames_puuid = data['metadata']['participants']
    usernames = []
    for puuid in usernames_puuid:
        usernames.append(get_name_and_tag(puuid, api_key)[0])
    return usernames

def get_match_KDA(matchID, playerID, api_key):
    data = get_match_data(matchID, api_key)
    participant_puuids = data['metadata']['participants']
    player_idx = 0
    for i in range(10):
        if (participant_puuids[i]==playerID):
            player_idx = i
    player_info = data['info']['participants'][player_idx]
    return (player_info['kills']+player_info['assists'])/max(player_info['deaths'], 1)

def get_teamKDA(match_data, blue):
    start = 5
    if blue:
        start = 0
    participant_info = match_data['info']['participants']
    k = 0 
    d = 0
    a = 0
    for i in range(start, start+5):
        k += participant_info[i]['kills']
        d += participant_info[i]['deaths']
        a += participant_info[i]['assists']
    return (k+a)/max(d, 1)

def get_indivKDA(match_data):
    ret = np.zeros((10, 1))
    participant_info = match_data['info']['participants']
    for i in range(10):
        k = 0
        d = 0
        a = 0
        k += participant_info[i]['kills']
        d += participant_info[i]['deaths']
        a += participant_info[i]['assists']
        ret[i, 0] = (k+a)/max(d, 1)
    return ret

def get_player_KDA(gameName, tagLine, api_key):
    puuid = get_puuid_ign(gameName, tagLine, api_key)
    matches = get_match_history(puuid, api_key)
    total = 0
    for i in range(20):
        total += get_match_KDA(matches[i], puuid, api_key)
    return total/20
    

In [44]:
def get_teamKDA_data(api_key):
    data = np.zeros((2, 100))
    labels = np.zeros((1, 100))
    hits = 0
    counter = 0
    while hits < 100:
        matchID = f"NA1_{5000000000-7*counter}"
        match_data = get_match_data(matchID, api_key)
        if "status" in match_data:
            counter += 1
            continue
        data[0, hits] = get_teamKDA(match_data, True)
        data[1, hits] = get_teamKDA(match_data, False)
        if match_data['info']['teams'][0]['win']:
            labels[0, hits] = 1
        else:
            labels[0, hits] = -1
        hits += 1
        counter += 1
        print(hits)
    return data, labels

def get_indivKDA_data(api_key):
    data = np.zeros((10, 100))
    labels = np.zeros((1, 100))
    hits = 0
    counter = 0
    while hits < 100:
        matchID = f"NA1_{5000000000-7*counter}"
        match_data = get_match_data(matchID, api_key)
        if "status" in match_data:
            counter += 1
            continue
        data[:, hits:hits+1] = get_indivKDA(match_data)
        if match_data['info']['teams'][0]['win']:
            labels[0, hits] = 1
        else:
            labels[0, hits] = -1
        hits += 1
        counter += 1
        print(hits)
    return data, labels


In [45]:
data2, labels2 = get_indivKDA_data(api_key)
data1, labels1 = get_teamKDA_data(api_key)
#test = get_match_data("NA1_5002919712", api_key)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [51]:
def averaged_perceptron(data, labels):
    d, n = data.shape
    theta = np.zeros((d, 1))
    theta0 = np.zeros((1, 1))

    theta_sum = theta.copy()
    theta0_sum = theta0.copy()

    for t in range(10):
        for i in range(n):
            x = data[:, i:i+1]
            y = labels[:, i:i+1]
            sign = np.sign(y*(theta.T@x+theta0))[0, 0]
            if sign!=1:
                theta = theta + y*x
                theta0 = theta0 + y
            theta_sum += theta
            theta0_sum += theta0

    theta = theta_sum/(10*n)
    theta0 = theta0_sum/(10*n)
    return theta, theta0

def score(data_test, labels_test, th, th0):
    _, n = data_test.shape
    hits = 0
    for i in range(n):
        data = data_test[:, i:i+1]
        if (labels_test[0, i:i+1] == np.sign(data.T @ th + th0))[0, 0]:
            hits += 1
    return hits/n

def eval(data_train, labels_train, data_test, labels_test):
    th, th0 = averaged_perceptron(data_train, labels_train)
    return score(data_test, labels_test, th, th0)

def xval_learning_alg(data, labels, k):
    d, _ = data.shape
    merged_datalabels = np.concatenate((data, labels))
    merged_datalabels = merged_datalabels.T
    np.random.shuffle(merged_datalabels)
    merged_datalabels = merged_datalabels.T
    data = merged_datalabels[0:d, :]
    labels = merged_datalabels[d:d+1, :]

    split_data = np.array_split(data, k, axis=1)
    split_labels = np.array_split(labels, k, axis=1)
    score = 0
    for i in range(k):
        data_train = np.concatenate(split_data[:i]+split_data[i+1:], axis=1)
        labels_train = np.concatenate(split_labels[:i]+split_labels[i+1:], axis=1)
        data_test = split_data[i]
        labels_test = split_labels[i]
        score += eval(data_train, labels_train, data_test, labels_test)
    return score/k        
    
        

In [52]:
print(xval_learning_alg(data1, labels1, 10))
print(xval_learning_alg(data2, labels2, 10))

0.8300000000000001
0.8099999999999999


In [53]:
from statsmodels.stats.proportion import proportions_ztest
proportions_ztest(np.array([83, 81]), np.array([100, 100]), 0)

(0.3681050869161534, 0.7127948779904199)